In [1]:
import tensorflow as tf
import numpy as np
tf.__version__

'2.0.0'

In [2]:
import zipfile

In [3]:
def load_data_jay_lyrics():
    """Load the Jay Chou lyric data set (available in the Chinese book)."""
    with zipfile.ZipFile('../../data/jaychou_lyrics.txt.zip') as zin:
        with zin.open('jaychou_lyrics.txt') as f:
            corpus_chars = f.read().decode('utf-8')
    corpus_chars = corpus_chars.replace('\n', ' ').replace('\r', ' ')
    corpus_chars = corpus_chars[0:10000]
    idx_to_char = list(set(corpus_chars))
    char_to_idx = dict([(char, i) for i, char in enumerate(idx_to_char)])
    vocab_size = len(char_to_idx)
    corpus_indices = [char_to_idx[char] for char in corpus_chars]
    return corpus_indices, char_to_idx, idx_to_char, vocab_size

In [4]:
corpus_indices, char_to_idx, idx_to_char, vocab_size = load_data_jay_lyrics()

In [5]:
vocab_size

1027

## 简介

现在我们考虑输入数据存在时间相关性的情况。假设$\boldsymbol{X}_t \in \mathbb{R}^{n \times d}$是序列中时间步$t$的小批量输入,$\boldsymbol{H}_t \in \mathbb{R}^{n \times d}$是该时间步的隐藏变量。与多层感知机不同的是，这里我们保存上一时间步的隐藏变量$\boldsymbol{H}_{t-1}$，并引入一个新的权重参数$\boldsymbol{W}_{hh} \in \mathbb{R}^{h \times h}$，该参数用来描述在当前时间步如何使用上一时间步的隐藏变量。具体来说，时间步$t$的隐藏变量的计算由当前时间步的输入和上一时间步的隐藏变量共同决定：
$$\boldsymbol{H}_t = \phi(\boldsymbol{X}_t \boldsymbol{W}_{xh} + \boldsymbol{H}_{t-1} \boldsymbol{W}_{hh}  + \boldsymbol{b}_h).$$

与多层感知机相比，我们在这里添加了$\boldsymbol{H}_{t-1} \boldsymbol{W}_{hh} $ 一项。由上式中相邻时间步的隐藏变量$\boldsymbol{X}_t$和$\boldsymbol{X}_{t-1}$之间的关系可知，这里的隐藏变量能够捕捉截至当前时间步的序列的历史信息，就像是神经网络当前时间步的状态或记忆一样。因此，该隐藏变量也称为隐藏状态。由于隐藏状态在当前时间步的定义使用了上一时间步的隐藏状态，上式的计算是循环的。使用循环计算的网络即循环神经网络（recurrent neural network）。

循环神经网络有很多种不同的构造方法。含上式所定义的隐藏状态的循环神经网络是极为常见的一种。若无特别说明，本章中的循环神经网络均基于上式中隐藏状态的循环计算。在时间步$t$，输出层的输出和多层感知机中的计算类似：
$$\boldsymbol{O}_t = \boldsymbol{H}_t \boldsymbol{W}_{hq} + \boldsymbol{b}_q.$$

循环神经网络的参数包括隐藏层的权重 $\boldsymbol{W}_{xh} \in \mathbb{R}^{d \times h}$、 $\boldsymbol{W}_{hh} \in \mathbb{R}^{h \times h}$ 和偏差  $\boldsymbol{b}_h \in \mathbb{R}^{1 \times h}$ ，以及输出层的权重 $\boldsymbol{W}_{hq} \in \mathbb{R}^{h \times q}$ 和偏差 $\boldsymbol{b}_q \in \mathbb{R}^{1 \times q}$。值得一提的是，即便在不同时间步，循环神经网络也始终使用这些模型参数。因此，循环神经网络模型参数的数量不随时间步的增加而增长。

In [6]:
x,w_xh = tf.random.normal(shape=(3,2)),tf.random.normal(shape=(2,4))
h,w_hh = tf.random.normal(shape=(3,4)),tf.random.normal(shape=(4,4))
b_h = tf.random.normal(shape=(1,4))

In [7]:
h_t = tf.matmul(x,w_xh)+tf.matmul(h,w_hh)
h_t

<tf.Tensor: id=32, shape=(3, 4), dtype=float32, numpy=
array([[ 0.9330221 , -1.7209074 , -2.7449512 ,  2.4443598 ],
       [ 2.6006992 ,  1.5038726 , -0.73471355,  0.32491297],
       [-1.0247426 , -1.8536962 , -2.6441722 ,  2.626462  ]],
      dtype=float32)>

In [8]:
tf.matmul(tf.concat([x,h],axis=1),tf.concat([w_xh,w_hh],axis=0))

<tf.Tensor: id=37, shape=(3, 4), dtype=float32, numpy=
array([[ 0.9330221, -1.7209076, -2.7449512,  2.4443598],
       [ 2.6006992,  1.5038726, -0.7347137,  0.3249129],
       [-1.0247426, -1.8536962, -2.6441724,  2.626462 ]], dtype=float32)>

## OneHot编码

In [9]:
tf.one_hot([0,2],depth=vocab_size)

<tf.Tensor: id=42, shape=(2, 1027), dtype=float32, numpy=
array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]], dtype=float32)>

In [10]:
def to_onehot(X,depth):
    outs = [tf.one_hot(x,depth=depth) for x in tf.transpose(X)]
    return tf.stack(outs,axis=0)

In [11]:
x = np.array(range(10)).reshape((2,5))
to_onehot(x,vocab_size)

<tf.Tensor: id=86, shape=(5, 2, 1027), dtype=float32, numpy=
array([[[1., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 1., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]], dtype=float32)>

In [12]:
def rnn(inputs,params,state):
    W_xh, W_hh, b_h, W_hq, b_q = params
    H=state
    outputs = []
    for x in inputs:
        a = tf.matmul(x,W_xh)
        b = tf.matmul(H,W_hh)
        H = tf.tanh( a + b + b_h)
        outputs.append( tf.keras.activations.softmax(tf.matmul(H,W_hq) + b_q))
        
    return tf.stack(outputs),H

In [13]:
num_inputs, num_hiddens, num_outputs = vocab_size, 256, vocab_size

In [14]:
def init_params():

    W_xh = tf.random.normal(shape=(num_inputs,num_hiddens))
    W_hh = tf.random.normal(shape=(num_hiddens,num_hiddens))
    b_h = tf.zeros(shape=(1,num_hiddens))

    W_hq = tf.random.normal(shape=(num_hiddens,num_outputs))
    b_q = tf.zeros(shape=(1,num_outputs))
    
    return W_xh,W_hh,b_h,W_hq,b_q

In [15]:
def init_rnn_state(batch_size, num_hiddens):
    return tf.zeros(shape=(batch_size, num_hiddens)) 

In [16]:
state = init_rnn_state(x.shape[0],num_hiddens)

In [17]:
inputs = to_onehot(x,vocab_size)
rnn(inputs,init_params(),state)

(<tf.Tensor: id=218, shape=(5, 2, 1027), dtype=float32, numpy=
 array([[[1.0269333e-15, 5.2755109e-13, 1.0907504e-12, ...,
          2.5514712e-19, 3.9393851e-16, 4.4626074e-19],
         [1.2050021e-19, 3.8631719e-14, 1.4777405e-19, ...,
          2.8166343e-09, 7.3760730e-15, 4.5071572e-15]],
 
        [[4.6540255e-21, 1.6330873e-21, 8.4880560e-32, ...,
          7.7777580e-19, 1.5149020e-26, 9.5517963e-31],
         [3.2947158e-23, 1.4410954e-13, 4.7374664e-22, ...,
          3.4390364e-27, 2.4437146e-29, 1.2373804e-21]],
 
        [[6.9397681e-17, 8.7288145e-27, 4.3863918e-21, ...,
          1.2644458e-20, 1.0586329e-15, 5.1568602e-18],
         [1.6680184e-24, 4.1121730e-20, 2.4531889e-19, ...,
          2.9208754e-06, 2.0136901e-11, 8.0009505e-10]],
 
        [[6.2488128e-31, 1.6814829e-12, 5.6531156e-22, ...,
          1.8573003e-22, 3.1143443e-12, 1.9550581e-15],
         [3.7855802e-17, 4.4262974e-06, 2.5365010e-17, ...,
          4.3525763e-31, 3.7682961e-14, 3.9637309e-16]],

In [18]:
def predict_rnn(prefix, num_chars, rnn, params, init_rnn_state,
                num_hiddens, vocab_size, idx_to_char, char_to_idx):
    state = init_rnn_state(1, num_hiddens)
    output = [char_to_idx[prefix[0]]]
    for t in range(num_chars + len(prefix) - 1):
        # 将上一时间步的输出作为当前时间步的输入
        X = to_onehot([output[-1]], vocab_size)
        X = tf.expand_dims(X,axis=0)
        # 计算输出和更新隐藏状态
        (Y, state) = rnn(X, params,state )
        # 下一个时间步的输入是prefix里的字符或者当前的最佳预测字符
        if t < len(prefix) - 1:
            output.append(char_to_idx[prefix[t + 1]])
        else:
            output.append(int(tf.argmax(Y[0],axis=1).numpy()))
    return ''.join([idx_to_char[i] for i in output])

In [19]:
predict_rnn('分开', 10, rnn, init_params(), init_rnn_state, num_hiddens, vocab_size, idx_to_char, char_to_idx)

'分开痛故肩打抢骷已育肩钩'

In [20]:
import jieba

In [21]:
with zipfile.ZipFile('../../data/jaychou_lyrics.txt.zip') as zin:
    with zin.open('jaychou_lyrics.txt') as f:
        corpus_chars = f.read().decode('utf-8')
corpus_chars = corpus_chars.replace('\n', '').replace('\r', '')

tokens = jieba.cut(corpus_chars)

In [22]:
seg_list = []
for i in tokens:
    seg_list.append(i)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\Administrator\AppData\Local\Temp\jieba.cache
Loading model cost 0.687 seconds.
Prefix dict has been built succesfully.


In [23]:
#生成onehot
vocab = sorted(list(set(seg_list)))
word_to_int = dict((w,i) for i,w in enumerate(vocab))
int_to_word = dict((i,w) for i,w in enumerate(vocab))

In [24]:
len(vocab),int_to_word[2475],word_to_int["我"]

(5727, '我', 2475)

In [25]:
n_words = len(seg_list)
vocab_size = len(vocab)
n_words,vocab_size

(37434, 5727)

In [26]:
seq_lenght = 100
dataX = []
dataY = []
for i in range(0,n_words-seq_lenght,1):
    seq_in = seg_list[i:i+seq_lenght+1]
    dataX.append([word_to_int[word] for word in seq_in])

np.random.shuffle(dataX)
for i in range(len(dataX)):
    dataY.append([dataX[i][seq_lenght]])
    dataX[i] = dataX[i][:seq_lenght]

In [27]:
len(dataX[0])

100

In [28]:
tf.one_hot(dataX[0],vocab_size)

<tf.Tensor: id=607, shape=(100, 5727), dtype=float32, numpy=
array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]], dtype=float32)>

In [141]:
n_simples = len(dataX)
print('样本数量：', n_simples)
X = np.reshape(dataX, (n_simples, seq_lenght))
y = tf.keras.utils.to_categorical(dataY)

样本数量： 37334


In [147]:
temp = ""
for ind in X[0]:
    temp += int_to_word[ind] 
temp

'岁月我用无悔 刻永世爱你的碑啦儿啦 啦儿啦 啦儿啦儿啦啦儿啦 啦儿啦 啦儿啦儿啦铜镜映无邪 紮马尾你若撒野 今生我把酒奉陪啦儿啦 啦儿啦 啦儿啦儿啦啦儿啦 啦儿啦 啦儿啦儿啦铜镜映无邪 紮马尾你若撒野 今生我把酒奉陪一件黑色毛衣两个人的回忆雨过之后更难'

In [149]:
model = tf.keras.Sequential([
    
    tf.keras.layers.Embedding(vocab_size,16,input_length=seq_lenght),
    tf.keras.layers.SimpleRNN(128),
    tf.keras.layers.Dense(vocab_size,activation="softmax"),
])

adam = tf.keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model.compile(loss='categorical_crossentropy', optimizer=adam)

In [150]:
model.fit(X, y, epochs=5, batch_size=64)

Train on 37334 samples
Epoch 1/5
37334/37334 [==============================] - 22s 592us/sample - loss: 7.0411
Epoch 2/5
37334/37334 [==============================] - 20s 541us/sample - loss: 6.3995
Epoch 3/5
37334/37334 [==============================] - 20s 543us/sample - loss: 5.7953
Epoch 4/5
37334/37334 [==============================] - 20s 541us/sample - loss: 5.2082
Epoch 5/5
37334/37334 [==============================] - 20s 542us/sample - loss: 4.6887


In [151]:
word_to_int["分开"]

949

In [155]:
# 生成种子
start = np.random.randint(0, len(dataX) - 1)
pattern = dataX[start]
print("Seed : ")
print(''.join([int_to_word[value] for value in pattern]))
n_generation = 200  # 生成的长度
print('开始生成，生成长度为', n_generation)
finall_result = []
for i in range(n_generation):
    x = np.reshape(pattern, (1, len(pattern)))
    prediction = model.predict(x, verbose=0)[0]
    index = np.argmax(prediction)
    result = int_to_word[index]
    # sys.stdout.write(result)
    finall_result.append(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]

# print(finall_result)
for i in range(len(finall_result)):
    if finall_result[i] != '。':
        print(finall_result[i], end='')
    else:
        print('。')

Seed : 
霓虹 不安跳动 染红夜空过去种种 象一场梦 不敢去碰 一想就痛心碎内容 每一秒钟 都有不同 你不懂连一句珍重 也有苦衷 也不想送寒风中 废墟烟囱 停止转动 一切落空在人海中 盲目跟从 别人的梦 全面放纵恨没有用 疗伤止痛 不再感动 没有梦痛不知轻重 泪水鲜红 全面放纵恨自己真的没用 情绪激动一颗心到现在还在抽痛
开始生成，生成长度为 200
 在一种味道叫做家没法挑剔 的窝 我的风火轮 的等候 我中的愿望     你的爱 我会  我的兄弟 放开 使用双截棍 哼哼哈兮快使用双截棍 哼哼哈兮快使用双截棍 哼哼哈兮习武之人切记 仁者无敌   很解药     我的兄弟 放开了窗玻璃     你在爱情 而   我不说 我           喔 喔 喔 哎哟 喔 哎哟 喔 哎哟 喔 哎哟 喔 哎哟 喔 哎哟 喔 哎哟 喔 哎哟 喔 哎哟 喔 哎哟 喔 哎哟 喔 哎哟 喔 哎哟喔喔 啦儿啦啦啦啦啦啦啦啦啦啦啦啦啦啦啦啦啦啦啦啦啦啦啦啦啦啦啦啦啦啦啦啦啦